In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

walker_60_suwon = pd.read_csv("D:\\jupyter_proj\\walker_60_suwon.csv")

In [2]:
Q1 = walker_60_suwon['도로심각도지수basic'].quantile(.25)
Q3 = walker_60_suwon['도로심각도지수basic'].quantile(.75)
Q2 = walker_60_suwon['도로심각도지수basic'].quantile(.5)
Q4 = walker_60_suwon['도로심각도지수basic'].quantile(1)

print("Q1 =",Q1)
print("Q2 =",Q2)
print("Q3 =",Q3)
print("Q4 =",Q4)

Q1 = 0.0068
Q2 = 0.1168
Q3 = 0.1168
Q4 = 1.0068


In [3]:
for i in range(len(walker_60_suwon)):
    if walker_60_suwon['도로심각도지수basic'][i] <= 0.0068:
        walker_60_suwon['도로심각도지수basic'][i] = '하'
    elif walker_60_suwon['도로심각도지수basic'][i] <= 0.1168:
        walker_60_suwon['도로심각도지수basic'][i] = '중'
    else:
        walker_60_suwon['도로심각도지수basic'][i] = '상'

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in walker_60_suwon.columns:
    if walker_60_suwon[column_name].dtype == object:
        walker_60_suwon[column_name] = le.fit_transform(walker_60_suwon[column_name])
    else:
        pass

In [9]:
xval = walker_60_suwon[walker_60_suwon.columns.difference(['도로심각도지수basic'])]
yval = walker_60_suwon['도로심각도지수basic']

x_train, x_test, y_train, y_test = train_test_split(xval, yval, train_size=0.7, random_state=97)
pipeline = Pipeline([('clf', RandomForestClassifier(criterion='gini', class_weight = 'balanced'))])

In [10]:
parameters = {
    'clf__n_estimators':(100, 2000, 3000),
    'clf__max_depth':(5, 15, 30),
    'clf__min_samples_split':(2,3),
    'clf__min_samples_leaf':(1,2)
}

In [12]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=5, verbose=1, scoring='accuracy')
grid_search.fit(x_train, y_train)
    

print('Best Training score : %0.3f' %grid_search.best_score_)
print('Best parameter set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' %(param_name, best_parameters[param_name]))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   59.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  5.0min finished


Best Training score : 0.540
Best parameter set:
	clf__max_depth: 15
	clf__min_samples_leaf: 1
	clf__min_samples_split: 3
	clf__n_estimators: 100


In [14]:
from sklearn.metrics import accuracy_score, classification_report

predictions = grid_search.predict(x_test)

print("Testing accuracy:", round(accuracy_score(y_test, predictions),4))
print("\nComplete report of Testing data\n", classification_report(y_test, predictions))

Testing accuracy: 0.5469

Complete report of Testing data
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.60      0.59      0.60       237
           2       0.49      0.57      0.52       174

    accuracy                           0.55       437
   macro avg       0.36      0.39      0.37       437
weighted avg       0.52      0.55      0.53       437



C:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
print('\n\nRandom Forest Grid Search- Test Confusion Matrix\n\n', pd.crosstab(y_test, predictions, rownames=["Actuall"],colnames=["Predicted"]))



Random Forest Grid Search- Test Confusion Matrix

 Predicted    1   2
Actuall           
0           18   8
1          140  97
2           75  99


In [17]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predictions)

array([[  0,  18,   8],
       [  0, 140,  97],
       [  0,  75,  99]], dtype=int64)